In [1]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_2f5959ebff3e42af80f2f5319da6bf35 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_2f5959ebff3e42af80f2f5319da6bf35 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_2f5959ebff3e42af80f2f5319da6bf35 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='TyQhUuTU4WIFu8GpdomRCKl-xaOYzuOXlGwJi94bcYZE',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_2f5959ebff3e42af80f2f5319da6bf35)

body = client_2f5959ebff3e42af80f2f5319da6bf35.get_object(Bucket='courserasubmission-donotdelete-pr-pz8fhzlrpe5khh',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1 = df_data_1.rename(columns={'Postal Code':'Postal_Code', 'Latitude':'Latitude','Longitude':'Longitude'})
df_data_1.columns
#df_data_1.head()


#This Code is for the Week 3 project for the capstone course.  In this first section, we acquire the Toronto Neighborhood data from wikipedia, and clean it up.

Index(['Postal_Code', 'Latitude', 'Longitude'], dtype='object')

In [2]:
!pip install requests-html
import pandas as pd
from bs4 import BeautifulSoup
from requests_html import HTMLSession
session = HTMLSession()

html_doc = session.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") 
html_doc = html_doc.text
#html_doc
soup = BeautifulSoup(html_doc, 'html.parser')

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    #print(row)
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [4]:
# initialize your variable to None
latitudination = []
longitudination = []

for postal_code_1 in df['PostalCode']:
    for index,row in df_data_1.iterrows():
        #print(row['Postal_Code'])
        if(postal_code_1 == row['Postal_Code']):
            latitude = row['Latitude']
            longitude = row['Longitude']
            continue
    
    latitudination.append(latitude)
    longitudination.append(longitude)
df['Latitude'] = latitudination
df['Longitude'] = longitudination
df.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [5]:
df.shape

(103, 5)